In [2]:
from cryptography.hazmat.primitives.serialization import load_pem_public_key, load_pem_private_key, BestAvailableEncryption
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.serialization import load_pem_public_key

def add_padding(msg: bytes, m: int) -> bytes:
    pad_length = (m - len(msg) % m) if (len(msg) % m) else 0
    padding = b'\x00' * pad_length \
            + b'\x01' * (m - pad_length) \
            + b'\x00' * pad_length
    return msg + padding

def remove_padding(msg: bytes, m: int) -> bytes:
    pad_length = m - sum(msg[-m:])
    return msg[:-m-pad_length]


In [7]:
def rsa_encode(
    src_path: str, 
    key_path: str, 
    dst_path: str,
) -> None:
    # Load the message's bytes
    with open(src_path, 'r') as file:
        msg = add_padding(str.encode(file.read()), 190)
    print(msg)
    # Load the public key
    with open(key_path, 'rb') as file:
        public_key = load_pem_public_key(file.read())
    # Encrypt the message
    buffer = b''
    while msg:
        buffer += public_key.encrypt(
            msg[:190],
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None))
        msg = msg[190:]
    # Save the encrypted message
    with open(dst_path, 'wb') as file:
        file.write(buffer)

rsa_encode('msg.txt', 'paris_public.key', 'msg.asy')

b'Le Tupolev-114 (en russe : \xc3\x90\xc2\xa2\xc3\x91\xc6\x92\xc3\x90\xc2\xbf\xc3\x90\xc2\xbe\xc3\x90\xc2\xbb\xc3\x90\xc2\xb5\xc3\x90\xc2\xb2-114) est un avion de ligne sovi\xc3\x83\xc2\xa9tique \xc3\x83\xc2\xa0 quatre turbopropulseurs construit par Tupolev dans les ann\xc3\x83\xc2\xa9es 1950 et produit \xc3\x83\xc2\xa0 32 exemplaires. \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [9]:
def rsa_decode(
    src_path: str,
    key_path: str,
    dst_path: str,
    passwd: str,
) -> None:
    # Load the private key
    with open(key_path, 'rb') as file:
        private_key = load_pem_private_key(
            file.read(),
            str.encode(passwd))
    # Load the encrypted bytes
    with open(src_path, 'rb') as file:
        message = file.read()
    # Decrypt the message
    buffer = b''
    while message:
        buffer += private_key.decrypt(
            message[:256],
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )
        message = message[256:]
    # Save the decrpyted message
    buffer = remove_padding(buffer, 190)
    with open(dst_path, 'w') as file:
        x = buffer.decode('utf-8')
        file.write(x)

rsa_decode('msg.asy', 'paris_private.key', 'decrypted.txt', 'secret')    